# Examples for using the RR Lyrae LSST griz templates by [Braga+ 2024]

This notebook will guide the user in all the possible cases in which it is possible to apply the RR Lyrae LSST templates. The aim is always to derive a mean magnitude from

1) A few magnitude measurements in LSST (see details for each case)

2) The knowledge of pulsation properties from other data (see details for each case)

In [1]:
# Note that RV_template_function imports the module mpfit, 
# which is a third-party product, available within astrolibpy
import apply_ugrizy_templates
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from PyPDF2 import PdfMerger
import os

First of all, set up the folder in which you will be working

In [2]:
folder = os.getcwd()+'/' #To be changed by the user
folder_coefficient_table = os.getcwd()+'/' #To be changed by the user
folder_rrls = folder_coefficient_table+'test/' #To be changed by the user
folder_coeff = folder #To be eventually changed by the user
folder_rvcs = folder_rrls+'rvcs/'
file_coeff = folder+'templates_analytics_230901.csv'

The following three cases will be discussed

1) When only one mag measurement is available. The template must be anchored to the point.

2) When two mag measurements,are available. In this case, the technique to adopt is identical. The template must be anchored to the points, recursively, and the mean magnitude should be derived as the average mean magnitudes of the two LCTs.

3) When three or more mag measurements are available. The template must be used as a fitting function with two free parameters (DeltaPhase, DeltaMag).

4) When four or more mag measurements are available. The template must be used as a fitting function with three free parameters (DeltaPhase, DeltaMag, Ampl).

## Case 1: One magnitude measurement available

In this case, it is mandatory to know the full pulsation properties of the variable: Period, amplitude and epoch of reference. The templates are anchored to the epoch of mean magnitude on the rising branch ($t_{mean}^{ris}$) but we provide also a relation in case only the more classic epoch of maximum light was available ($t_{max}$).

In [3]:
pulsation_period = 0.59031000
t0 = 2456423.8564 # Epoch of the mean magnitude on the rising branch
pulsation_type = 0
amplitude_for_template = 0.65
passband = 3

hjd = np.asarray([2457116.7218])
mag = np.asarray([15.6200])
err = np.asarray([0.0140])
phase = ((hjd - t0)/pulsation_period) % 1

templatebin_int = apply_ugrizy_templates.find_templatebin(pulsation_type, 
                                                          pulsation_period, 
                                                          passband)[0]

result_resample = apply_ugrizy_templates.anchor_template(hjd, mag, err, pulsation_type,
                                    pulsation_period, t0, passband, amplitude_for_template, 
                                    file_coeff, figure_out=folder+'test1.pdf')

print('Mean mag: {:6.3f}+-{:5.3f}'.format(result_resample['mean_mag_mean'], result_resample['mean_mag_err']))

Mean mag: 15.443+-0.014


## Case 2: Two magnitude measurements available

In this case, it is mandatory to know the full pulsation properties of the variable: Period, amplitude and epoch of reference. The templates are anchored to the epoch of mean magnitude on the rising branch ($t_{mean}^{ris}$) but we provide also a relation in case only the more classic epoch of maximum was available ($t_{max}$).

In [4]:
pulsation_period = 0.59031000
t0 = 2456423.8564 # Epoch of the mean magnitude on the rising branch
pulsation_type = 0
amplitude_for_template = 0.65
passband = 3

hjd = np.asarray([2457116.7218, 2456453.8418])
mag = np.asarray([15.6200, 15.5830])
err = np.asarray([0.0140, 0.0130])
phase = ((hjd - t0)/pulsation_period) % 1

templatebin_int = apply_ugrizy_templates.find_templatebin(pulsation_type, 
                                                          pulsation_period, 
                                                          passband)[0]

result_resample = apply_ugrizy_templates.anchor_template(hjd, mag, err, pulsation_type,
                                    pulsation_period, t0, passband, amplitude_for_template, 
                                    file_coeff, figure_out=folder+'test2.pdf')

print('Mean mag: {:6.3f}+-{:5.3f}'.format(result_resample['mean_mag_mean'], result_resample['mean_mag_err']))

Mean mag: 15.418+-0.036


In [5]:
## METTERE IN UNA FUNZIONE

if pulsation_type == 0:
    amplmaxs = {2: 1.8, 3: 1.5, 4: 1.1, 5: 1.1}
    amplmins = {2: .2, 3: .15, 4: .1, 5: .1}
    amplratio_ogle_to_use = 'Ampl(x)/Ampl(I)_ab'
else:
    amplmaxs = {2: .9, 3: .7, 4: .55, 5: 1.1}
    amplmins = {2: .1, 3: .05, 4: 0., 5: 0.}    
    amplratio_ogle_to_use = 'Ampl(x)/Ampl(I)_c'

# # XXX Visto che ho dimostrato l'uguaglianza tra ZTF e LSST, posso prendere amplratio_ZTF_OGLE
# df_amplratios_LSST_OGLE = pd.read_csv(folder+'amplratio_ZTF_OGLE.csv')

# # Fixed amplitude
#         result_resample = apply_ugrizy_templates.apply_templatefit(hjd, mag, err, pulsation_type,
#                                            pulsation_period, t0_temp, passband,
#                                             file_coeff, free_amplitude=False, ampl_to_use)

# # # Free amplitude
# # result_resample = apply_ugrizy_templates.apply_templatefit(hjd, 
# #                                     mag, err, pulsation_type,
# #                                     pulsation_period, t0_temp, passband,
# #                                     file_coeff, ampl=(amplmaxs[passband]+amplmins[passband])/2.,
# #                                     amplmax=amplmaxs[passband], amplmin=amplmins[passband])

## Case 3: Three or more magnitude measurements available

When three or more mag measurements are available for one target, it is possible to use the template not by anchoring it to a given epoch, but as a fitting function, leaving two free parameters: the phase displacement and the mean mag displacement. In this case, only Period, and amplitude are needed as previous knowledge. Note that the function apply_template_templfit_amplfixed takes as input t0, but this can be arbitrary and only affects visualization and not the results.

In [6]:
pulsation_period = 0.59031000
t0 = 2456423.8564 # Epoch of the mean magnitude on the rising branch
pulsation_type = 0
amplitude_for_template = 0.65
passband = 3

hjd = np.asarray([2457116.7218, 2457114.7496, 2457115.8305])
mag = np.asarray([15.6200, 15.4260, 15.1980])
err = np.asarray([0.0140, 0.0130, 0.0140])
phase = ((hjd - t0)/pulsation_period) % 1

templatebin_int = apply_ugrizy_templates.find_templatebin(pulsation_type, 
                                                          pulsation_period, 
                                                          passband)[0]
result_resample = apply_ugrizy_templates.apply_templatefit(hjd, mag, err,
                                    pulsation_type, pulsation_period, t0, passband, 
                                    file_coeff, free_amplitude=False, 
                                    ampl=amplitude_for_template, 
                                    figure_out=folder+'test3.pdf')

print('Mean mag: {:6.3f}+-{:5.3f}'.format(result_resample['mag_mean'], result_resample['errmag_mean']))

Mean mag: 15.476+-0.009


## Case 4: Four or more magnitude measurements available

When three or more mag measurements are available for one target, it is possible to use the template not by anchoring it to a given epoch, but as a fitting function, leaving three free parameters: the phase displacement, the mean mag displacement and the amplitude. In this case, only Period, is needed as previous knowledge.

In [22]:
pulsation_period = 0.59031000
t0 = 2456423.8564 # Epoch of the mean magnitude on the rising branch
pulsation_type = 0
amplitude_for_template = 0.65
passband = 3

hjd = np.asarray([2457116.7218, 2457114.7496, 2457115.8305, 2457116.8515])
mag = np.asarray([15.6200, 15.4260, 15.1980, 15.6980])
err = np.asarray([0.0140, 0.0130, 0.0140, 0.0140])
phase = ((hjd - t0)/pulsation_period) % 1

templatebin_int = apply_ugrizy_templates.find_templatebin(pulsation_type, 
                                                          pulsation_period, 
                                                          passband)[0]
result_resample = apply_ugrizy_templates.apply_templatefit(hjd, mag, err,
                                    pulsation_type, pulsation_period, t0, passband, 
                                    file_coeff, ampl=(amplmaxs[passband]+amplmins[passband])/2.,
                                    amplmax=amplmaxs[passband], amplmin=amplmins[passband], 
                                    figure_out=folder+'test4.pdf')

print('Mean mag: {:6.3f}+-{:5.3f}'.format(result_resample['mag_mean'], result_resample['errmag_mean']))

Mean mag: 15.444+-0.007
